# Подключение к API LLM

Вам наверняка захочется использовать какие-то модели через API, здесь вы найдете сниппеты которые вы можете просто скопировать в свой код и обращаться к моделькам.

In [1]:
!pip install -r requirements.txt

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.2 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.m

## YandexGPT

In [2]:
import requests

iam_token = 't1.9euelZqKxpvKzJyTzMuTmJvMms7OjO3rnpWaj5uSm8eal5nHlMrGjMiLx83l8_c9ZlZD-e9zAHsM_t3z930UVEP573MAewz-zef1656VmseMmsjIz8vPzpSPiciZzJ6L7_zF656VmseMmsjIz8vPzpSPiciZzJ6L.gU-hI-l8zlbOU8XmrYXmo9l6iAoq8uKZp2WGM2n_Kd4tlnvWi-cwSToi2kB4ryptdQyoamICZzGtZaFTkyAcDA'
folder_id = 'b1ggrmkcatrn3fgllatd'
URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"

def run(iam_token, folder_id, user_text):
    # Собираем запрос
    data = {}
    # Указываем тип модели (https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models)
    data["modelUri"] = f"gpt://{folder_id}/yandexgpt-lite"
    # Настраиваем опции
    data["completionOptions"] = {"temperature": 0.3, "maxTokens": 1000}
    # Указываем контекст для модели
    data["messages"] = [
        {"role": "system", "text": user_text},
        {"role": "user", "text": f"{user_text}"},
    ]

    # Отправляем запрос
    response = requests.post(
        URL,
        headers={
            "Accept": "application/json",
            "Authorization": f"Bearer {iam_token}"
        },
        json=data,
    ).json()

    #Распечатываем результат
    print(response)

In [3]:
run(iam_token, folder_id, 'Чему кореферентно слово "она" в предложении "Мама(1) мыла раму(2), и тут она( ) упала."?')

{'result': {'alternatives': [{'message': {'role': 'assistant', 'text': 'В предложении «Мама мыла раму, и тут она упала» слово «она» кореферентно слову «мама».'}, 'status': 'ALTERNATIVE_STATUS_FINAL'}], 'usage': {'inputTextTokens': '69', 'completionTokens': '24', 'totalTokens': '93'}, 'modelVersion': '23.10.2024'}}


## OpenAI, Anthropic etc.

Названия моделей вы можете посмотреть [здесь](https://vsegpt.ru/Docs/Models)

Если у вас возникают какие-то проблемы с ключом/обращением к моделям пишите в ТГ: @andrey_bobr

In [6]:
# Using Langchain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini",
                   api_key="sk-or-vv-16f288696526c0df8f64ed2fba16048201564bac72ee4664ea6138642c527e7c",
                   base_url="https://api.vsegpt.ru/v1",
                   temperature=0)

print(model.invoke('Чему кореферентно слово "она" в предложении "Мама(1) мыла раму(2), и тут она( ) упала."?').content)

В предложении "Мама(1) мыла раму(2), и тут она( ) упала." слово "она" кореферентно слову "Мама". Это означает, что "она" ссылается на "Маму".


In [5]:
# Using OpenAI API directly
from openai import OpenAI

client = OpenAI(
    api_key="sk-or-vv-16f288696526c0df8f64ed2fba16048201564bac72ee4664ea6138642c527e7c",
    base_url="https://api.vsegpt.ru/v1",
)

prompt = 'Чему кореферентно слово "она" в предложении "Мама(1) мыла раму(2), и тут она( ) упала."?'

messages = []
messages.append({"role": "user", "content": prompt})

response_big = client.chat.completions.create(
    model="anthropic/claude-3-haiku", # id модели из списка моделей - можно использовать OpenAI, Anthropic и пр. меняя только этот параметр
    messages=messages,
    temperature=0.7
)

response = response_big.choices[0].message.content
print("Response:",response)


Response: В данном предложении слово "она" кореферентно (относится к тому же лицу или предмету) с "мама" (1).

Предложение можно разобрать следующим образом:

1. "Мама(1) мыла раму(2)" - действие выполняется мамой.
2. "и тут она(1) упала" - здесь "она" относится к маме (1), то есть мама упала.

Таким образом, слово "она" в этом предложении кореферентно слову "мама".
